# Практическая работа 1
## Тема: Загрузка данных
## Цель работы: 
Изучение технологии извлечения данных из различных источников: CSV-файлы, Excel-файлы, JSON-файлы, HTML-файлы, SQL-базы данных.

### Часть 1. Чтение CSV-файла

С использованием функции pd.read_csv() выполните следующие операции по чтению данных для датасета titanic.csv:

1. прочитайте файл и переведите его в датафрейм, в качестве индекса используйте столбец 'PassengerId';

2. извлеките из файла и сохраните в датафрейм первые 10 строк столбцов 'Name', 'Sex' и 'Survived';

3. прочитайте файл и задайте новый набор имен для столбцов в нижнем регистре;

4. прочитайте файл, пропустив первые 100 строк, а затем считайте следующие 8 строк;

5. прочитайте первые 10 строк из файла, пропустив строки 1, 3, 7.

In [24]:
import pandas as pd

# 1
pd.read_csv("titanic.csv", index_col="PassengerId")

# 2
pd.read_csv("titanic.csv", nrows=10, usecols=["Name", "Sex", "Survived"])

# 3
names = "passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked"
pd.read_csv("titanic.csv", header=0, names=names.split(","))

# 4
pd.read_csv("titanic.csv", skiprows=100, nrows=8)

# 5
pd.read_csv("titanic.csv", skiprows=[1, 3, 7], nrows=10)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
2,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
3,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
4,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
5,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
6,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
7,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
8,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
9,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S


### Часть 2. Чтение НТМL-файлов из Интернета

Изучив материал лекции, приведите свой пример на запись НТМL-таблицы в один или несколько объектов DataFгame. Продемонстрируйте возможность извлечения части таблицы.

In [ ]:
import pandas as pd
import webbrowser as web
from os import path


url = "https://www.cnews.ru/reviews/" \
"dbaas2023/review_table/5c3bf5dc1f2f4058c7bcc51019984c37c1acf3f3"

table, *_ = pd.read_html(url)
table = table.set_index("Место")

raiting = table.iloc[:5, :3]

raiting.to_html("rating.html")
path = path.abspath("rating.html")
web.open(f"file://{path}")


True

### Часть 3. Чтение и запись JSОN-файлов

Изучив материал лекции, приведите свой пример работы с JSОN-файлом.

In [43]:
import pandas as pd

rating, *_ = pd.read_html("rating.html")

raiting.to_json("rating.json", force_ascii=False, indent=4)
raiting = pd.read_json("rating.json")

raiting


,Провайдер,Сервис,Сумма баллов
1,Selectel,Облачные базы данных (Managed Databases),643
2,Yandex Cloud,Managed Service,639
3,VK Cloud,Cloud Databases,606
4,---,Облачные базы данных,541
5,Cloud,Relational Database Service,519


## Часть 4. Работа с базой данных SQL
1. Используя встроенную Python-библиотеку для работы с базами данных - sqlite3 создайте базу данных для хранения сведений о работе кафе со следующей структурой:

    ![Схема бд](schema.png)

    Самостоятельно подумайте, какие типы данных следует выбрать для полей таблиц.

2. Добавьте в таблицу "Блюда" 5 любых блюд, в таблицу "Заказы" - 3 заказа и в каждом заказе по 2-3 блюда (таблица "Заказано").

3. С помощью метода read_sql выполните извлечение данных из базы данных и представьте в виде датафрейма:

    - информацию о заказанных блюдах со следующими столбцами: номер заказа, название блюда, цена.
    - сумму к оплате для каждого заказа.
    - названия всех блюд, у которых цена меньше 300 руб.
    - все блюда заказа с номером 2 (или другим номером).
    - список блюд по убыванию цены.

    Измените расчёт суммы к оплате, предусмотрев скидку 5% при наличии в заказе 3 и более блюд.

4. Создайте датафреймы для хранения информации о трёх новых блюдах, двух новых заказах, в каждом из которых по 2-3 блюда. С помощью метода to_sql выполните запись в базу данных.

In [72]:
import sqlite3 as sl
from os import path
from subprocess import run
import pandas as pd

# 1 и 2
if not path.exists("cafe.db"):
    with open("init.sql", encoding="UTF-8") as f:
        run(["python", "-m", "sqlite3", "cafe.db"], stdin=f)


with sl.connect("cafe.db") as conn:

    # 3.1
    query = """
        select o.id as order_id, d.name as dish_name, d.price as dish_price
        from OrderItems as oi
        join Dishes as d on d.id = oi.dish_id
        join Orders as o on o.id = oi.order_id
    """

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)

    # 3.2
    query = """
        select o.id as order_id, sum(d.price) as total
        from OrderItems as oi
        join Dishes as d on d.id = oi.dish_id
        join Orders as o on o.id = oi.order_id
        group by o.id
    """

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)

    # 3.3
    query = "select name from dishes where price < 300"

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)

    # 3.4
    query = """
        select d.*
        from OrderItems as oi
        join Dishes as d on d.id = oi.dish_id
        join Orders as o on o.id = oi.order_id
        where o.id = 2
    """

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)

    # 3.5
    query = "select * from Dishes order by price desc"

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)


    # 3.6
    query = """
        select o.id as order_id, count(d.name) as count, sum(d.price) as total
        from OrderItems as oi
        join Dishes as d on d.id = oi.dish_id
        join Orders as o on o.id = oi.order_id
        group by o.id
    """

    items: pd.DataFrame = pd.io.sql.read_sql(query, conn)
    mask = items["count"] >= 3
    items.loc[mask, "total"] = (items.loc[mask, "total"] * (1 - 0.05)).round(2)


In [ ]:
import sqlite3 as sl
import pandas as pd


with sl.connect("cafe.db") as conn:

    # 4
    dishes_data = {
        "name": ["Бургер", "Суши", "Салат Оливье", "Ризотто с морепродуктами", "Шоколадный торт"],
        "price": [15.50, 20.00, 7.80, 18.25, 6.50]
    }
    dishes_df = pd.DataFrame(dishes_data)

    orders_data = {
        "date": ["2024-01-15", "2024-02-17", "2024-03-10"]
    }
    orders_df = pd.DataFrame(orders_data)

    order_items_data = {
        "order_id": [1, 1, 2, 2, 3, 3],
        "dish_id": [1, 4, 2, 5, 3, 4]
    }
    order_items_df = pd.DataFrame(order_items_data)

    dishes_df.to_sql("Dishes", conn, if_exists="append", index=False)
    orders_df.to_sql("Orders", conn, if_exists="append", index=False)
    order_items_df.to_sql("OrderItems", conn, if_exists="append", index=False)

#### Экспорт в html

In [75]:
from os import system

system("jupyter nbconvert --to html practice_1.ipynb")

[NbConvertApp] Converting notebook practice_1.ipynb to html
[NbConvertApp] Writing 303803 bytes to practice_1.html


0